In [ ]:
import pandas as pd
import numpy as np
#  python script
import sys; sys.path.append("/home/shhsu/download/scClass")
import scClass
from scClass.main import Model
from scClass import loom

In [ ]:
def dataset(path):
    return "/home/shhsu/download/scClass_bigdata/data_preprocessing/"+path
# data prepare
bm2_loom_norm = loom.read(dataset("bm2.loom"),dataset("bm3.npz"))
bm_table = pd.read_csv(dataset("label_table/BM_v2.csv"),keep_default_na=False)
cb2_loom_norm = loom.read(dataset("cb2.loom"),dataset("cb3.npz"))
cb_table = pd.read_csv(dataset("label_table/CB_v2.csv"),keep_default_na=False)
mix_table = pd.read_csv(dataset("label_table/type_id.csv"),keep_default_na=False)

In [ ]:
# get index
def get_index(adata,df,save=None):
    label_index = np.zeros(adata.obs["label"].shape[0], dtype=np.int8)
    for x in range(df["identity.text"].shape[0]):
        id_indexs = np.where(adata.obs["label"] == df["identity.text"][x])[0]
        for id_index in id_indexs:
            label_index[id_index] = df["index"][x]
    adata.obs["type_id"] = label_index        
    if save != None:
        adata.write_loom(save)
    return label_index

In [ ]:
bm_label_index = get_index(bm2_loom_norm,bm_table)
cb_label_index = get_index(cb2_loom_norm,cb_table)

In [ ]:
np.random.seed(10)
all_label_index = np.append(bm_label_index,cb_label_index)
for i in range(all_label_index.max()+1):
    k = np.random.permutation((all_label_index==i).nonzero()[0])[50_000:]
    all_label_index[k] = -1

In [ ]:
import matplotlib.pyplot as plt
x,y = np.unique(all_label_index,return_counts=True)
plt.bar(x,y)

In [ ]:
mix_label = np.empty((bm2_loom_norm.shape[0]+cb2_loom_norm.shape[0],2),dtype='O')
mix_label[:,0] = all_label_index
mix_label[:,1] = mix_table["level"][mix_label[:,0]+1]
scClass.write_csv(path="/home/shhsu/download/scClass_dataset/1M_immune_cells/data_train_v2_v2.csv",index=["index","label"],array=mix_label)